# 构建检索问答链

我们已经介绍了如何根据自己的本地知识文档，搭建一个向量知识库。 在接下来的内容里，我们将使用搭建好的向量数据库，对 query 查询问题进行召回，并将召回结果和 query 结合起来构建 prompt，输入到大模型中进行问答。   

## 1. 加载向量数据库

首先，我们加载在前一章已经构建的向量数据库。注意，此处你需要使用和构建时相同的 Emedding。

### Chroma

In [ ]:
# import sys
# sys.path.append("../C3 搭建知识库") # 将父目录放入系统路径中

from langchain.vectorstores.chroma import Chroma
from dotenv import load_dotenv, find_dotenv
import os
# # 从环境变量中加载你的 API_KEY
# _ = load_dotenv(find_dotenv())    # read local .env file
# zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

# 定义持久化目录
persist_directory = '../data_base/vector_db/chroma-vmax'

# # 创建嵌入模型
# from langchain_community.embeddings import ZhipuAIEmbeddings

# zhipu_embed = ZhipuAIEmbeddings(
#     model="embedding-2",
#     api_key=zhipuai_api_key
# )

from langchain_community.embeddings import OllamaEmbeddings
my_emb = OllamaEmbeddings(base_url='http://localhost:11434',model="bge-m3:latest")

try:
    # 加载持久化的 Chroma 向量数据库
    vectordb = Chroma(
        persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
        collection_name="vmax-s",
        embedding_function=my_emb
    )
    print("向量数据库已成功加载。")
except Exception as e:
    print(f"加载向量数据库时发生错误: {e}")

In [ ]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

In [ ]:
print(f"向量库中存储的数量：{vectordb._collection.count()}")

In [ ]:
question = "VMAX上网日志业务是什么？"
docs = vectordb.similarity_search(question,k=5)
print(f"检索到的内容数：{len(docs)}")

打印一下检索到的内容

In [ ]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

### Milvus

In [1]:
from langchain_community.vectorstores import Milvus
from langchain_community.embeddings import OllamaEmbeddings
my_emb = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")

# Milvus 连接参数
vectordb = Milvus(
        embedding_function=my_emb,
        collection_name="Vmaxs",  # Milvus 集合名称
        connection_args={
            "host": "192.168.0.188",  # Milvus 服务器地址
            "port": "19530",  # Milvus 默认端口
        },
    )

C:\Users\will\AppData\Local\Temp\ipykernel_3560\3633838239.py:3: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  my_emb = OllamaEmbeddings(base_url='http://localhost:11434', model="bge-m3:latest")
C:\Users\will\AppData\Local\Temp\ipykernel_3560\3633838239.py:6: LangChainDeprecationWarning: The class `Milvus` was deprecated in LangChain 0.2.0 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-milvus package and should be used instead. To use it run `pip install -U :class:`~langchain-milvus` and import as `from :class:`~langchain_milvus import MilvusVectorStore``.
  vectordb = Milvus(


In [2]:
results = vectordb.similarity_search(query="VMAX", k=2)
# results

## 2. 创建一个 LLM

在这里，我们调用 OpenAI 的 API 创建一个 LLM，当然你也可以使用其他 LLM 的 API 进行创建

In [3]:
from langchain_community.llms import Ollama

my_llm = Ollama(base_url='http://localhost:11434', model='qwen2.5:0.5b', temperature=0.1)

# my_llm.invoke("你好")

C:\Users\will\AppData\Local\Temp\ipykernel_3560\73905523.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  my_llm = Ollama(base_url='http://localhost:11434', model='qwen2.5:0.5b', temperature=0.1)


## 3. 构建检索问答链

prompts

In [ ]:
from langchain.prompts import PromptTemplate

template = """你是VMAX运维助手，使用以下上下文来回答问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)


#### 创建一个基于模板的检索链： 基础检索版本

In [ ]:
from langchain.chains import RetrievalQA

# 基础检索
base_retriever = vectordb.as_retriever(search_kwargs={"k": 10})
base_retriever = vectordb.as_retriever(
    search_kwargs={"k": 15},  # 扩大召回池
    search_type="mmr",  # 最大边际相关性算法（网页5）
    # metadata_filter={"source": "权威文档.pdf"}  # 元数据过滤
)

qa_chain = RetrievalQA.from_chain_type(my_llm,
                                       retriever=base_retriever,
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


## 4.检索问答链效果测试

### 4.1 基于召回结果和 query 结合起来构建的 prompt 效果

In [ ]:
question_1 = "什么是vmax的上网日志系统？"
result = qa_chain({"query": question_1})
print("大模型+知识库后回答 question_1 的结果：")
print(result["result"])

In [ ]:
question_2 = "严威是谁？"
result = qa_chain({"query": question_21})
print("大模型+知识库后回答 question_2 的结果：")
print(result["result"])

### 4.2 无知识库大模型自己回答的效果

In [ ]:
prompt_template = """请回答下列问题:
                            {}""".format(question_1)

### 基于大模型的问答
my_llm.predict(prompt_template)

In [ ]:
prompt_template = """请回答下列问题:
                            {}""".format(question_2)

### 基于大模型的问答
my_llm.predict(prompt_template)

> ⭐ 通过以上两个问题，我们发现 LLM 对于一些近几年的知识以及非常识性的专业问题，回答的并不是很好。而加上我们的本地知识，就可以帮助 LLM 做出更好的回答。另外，也有助于缓解大模型的“幻觉”问题。

## 5. 添加历史对话的记忆功能

现在我们已经实现了通过上传本地知识文档，然后将他们保存到向量知识库，通过将查询问题与向量知识库的召回结果进行结合输入到 LLM 中，我们就得到了一个相比于直接让 LLM 回答要好得多的结果。在与语言模型交互时，你可能已经注意到一个关键问题 - **它们并不记得你之前的交流内容**。这在我们构建一些应用程序（如聊天机器人）的时候，带来了很大的挑战，使得对话似乎缺乏真正的连续性。这个问题该如何解决呢？


### 5.1 记忆（Memory）

在本节中我们将介绍 LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。我们将使用 `ConversationBufferMemory` ，它保存聊天消息历史记录的列表，这些历史记录将在回答问题时与问题一起传递给聊天机器人，从而将它们添加到上下文中。

In [4]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",  # 与 prompt 的输入变量保持一致。
    return_messages=True  # 将以消息列表的形式返回聊天记录，而不是单个字符串
)

C:\Users\will\AppData\Local\Temp\ipykernel_3560\2228008247.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


关于更多的 Memory 的使用，包括保留指定对话轮数、保存指定 token 数量、保存历史对话的总结摘要等内容，请参考 langchain 的 Memory 部分的相关文档。

### 5.2 对话检索链（ConversationalRetrievalChain）

对话检索链（ConversationalRetrievalChain）在检索 QA 链的基础上，增加了处理对话历史的能力。

它的工作流程是:
1. 将之前的对话与新问题合并生成一个完整的查询语句。
2. 在向量数据库中搜索该查询的相关文档。
3. 获取结果后,存储所有答案到对话记忆区。
4. 用户可在 UI 中查看完整的对话流程。

这种链式方式将新问题放在之前对话的语境中进行检索，可以处理依赖历史信息的查询。并保留所有信
息在对话记忆中，方便追踪。

接下来让我们可以测试这个对话检索链的效果：

使用上一节中的向量数据库和 LLM ！首先提出一个无历史对话的问题“这门课会学习 Python 吗？”，并查看回答。

In [ ]:
# 新增：定制对话模板
from langchain.prompts import PromptTemplate

custom_template = """你是VMAX运维助手，基于以下对话历史和上下文知识，用中文回答用户的问题。
    历史对话记录：
    {chat_history}
    
    上下文知识：
    {context}
    
    当前问题：{question}
    
    回答要求：
    1. 如果问题需要专业领域知识，优先使用上下文内容
    2. 若答案不在知识库中，明确告知"根据已知信息无法回答"
    3. 结尾添加"是否需要进一步说明？"[2,7](@ref)
    """
    
# 创建包含变量占位的PromptTemplate
QA_PROMPT = PromptTemplate(
        input_variables=["chat_history", "context", "question"],
        template=custom_template
    )
    

In [6]:
# 新增：定制对话模板
from langchain.prompts import PromptTemplate
template = """你是VMAX运维助手，使用以下上下文来回答问题。如果你不知道答案，就说你不知道，不要试图编造答案。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template)

In [7]:
# from langchain.chains import ConversationalRetrievalChain

# retriever=vectordb.as_retriever()

# qa = ConversationalRetrievalChain.from_llm(
#     llm,
#     retriever=retriever,
#     memory=memory
# )
# question = "什么是VMAX？"
# result = qa({"question": question})
# print(result['answer'])

In [8]:
from langchain.chains import ConversationalRetrievalChain

retriever = vectordb.as_retriever(search_kwargs={"k": 5})  # 控制检索文档数量
    
# 修改链配置，注入自定义模板
qa = ConversationalRetrievalChain.from_llm(
        my_llm,
        retriever=retriever,
        memory=memory,
        combine_docs_chain_kwargs={"prompt": QA_CHAIN_PROMPT},  # 关键参数绑定模板
        get_chat_history=lambda h: h  # 保持历史记录原始格式[4](@ref)
    )
    

然后基于答案进行下一个问题“为什么这门课需要教这方面的知识？”：

In [ ]:
questions = [
    "什么是VMAX？",
    "VMAX有哪些功能", 
    "整理成表格"  
]

for question in questions:
    result = qa({"question": question})  # Pass string directly, not dict
    print(f"问题：{question}")
    print(f"回答：{result['answer']}")
    # print("对话历史：", memory.load_memory_variables({}))
    print("\n" + "="*50 + "\n")

In [11]:
questions = [
    "VMAX有哪些功能？",
    "整理成表格", 
]

for question in questions:
    result = qa({"question": question})  # Pass string directly, not dict
    print(f"回答：{result['answer']}")
    print("\n" + "="*50 + "\n")

回答：VMAX系统涉及的多个技术细节和技术问题包括：

1. **物理指标**：
   - 本章包含如下主题：物理指标（外形尺寸和重量）取决于具体项目的规模及选型的服务器。
   - 例如，ZXVMAX-S系统除三方服务器无其它硬件外，相应的物理指标（外形尺寸和重量）取决于具体项目的规模及选型的服务器。

2. **性能指标**：
   - ZXVMAX-S的性能指标包括用户数、系统最大配置输入流量、同时登录的用户/终端数等。
   - 例如，表8-1性能指标分类中提到的性能指标有：用户数#3000万、系统最大配置输入流量#300Gbps、同时登录的用户/终端数。

3. **功耗指标**：
   - 表8-1中的“功耗指标”部分没有具体提及，但通常包括CPU使用率、内存使用率等。
   - 例如，表8-1性能指标分类中提到的功耗指标有：KQI/KPI分析周期、原始数据存储时间、小时粒度数据表存储时间、365天探针数据采集量。

4. **时钟指标**：
   - 表8-1中的“时钟指标”部分没有具体提及，但通常包括系统运行时间等。
   - 例如，表8-1性能指标分类中提到的时钟指标有：KQI/KPI分析周期、原始数据存储时间、小时粒度数据表存储时间。

5. **应急或备份措施**：
   - 需要了解本运营商的相关标准和要求，并根据实际情况采取相应的应急或备份措施。
   - 例如，VMAX系统可能需要支持的应急或备份措施包括但不限于：设备紧急故障的判断、定位和排除方法、用户投诉渠道等。

6. **告警处理**：
   - VMAX多维价值分析系统告警处理部分涉及多个技术细节和技术问题，如4000200116S1-MME接口XDRID完整率(小时)。
   - 例如，该指标可能需要监控和管理以确保系统的稳定运行。

综上所述，VMAX系统涉及的多个技术细节和技术问题包括但不限于物理、性能、功耗、时钟、应急或备份措施以及告警处理。具体的技术细节和技术问题可能会根据实际情况有所不同。


回答：我是VMAX运维助手，阿里云提供的服务。如果您有任何关于VMAX的疑问或需要帮助的地方，请随时告诉我！




In [13]:
question="只保留前三个功能"
result = qa({"question": question})  
print(f"回答：{result['answer']}")

回答：VMAX主要提供以下功能：

1. 用户分析功能：
   - 以客户、客户群组等为维度的专项分析，可以区分业务、用户等级和时间等。
   - 可以进行用户级的在某一时间段的呼叫、注册记录、媒体数据、信令详情，并结合最近7天的统计情况定位语音/视频通话质量异常环节。

2. 一键投诉功能：
   - 输入用户号码或IMSI，以及用户投诉的起止时间。
   - 提供引起用户投诉的具体问题业务记录（即问题话单），并给出导致问题发生的原因。

3. 区域感知功能：
   - 定期对全网的所有小区进行排查，找到感知差的小区和原因分析。
   - 可以支持对事先配置的重点区域做感知分析，包括感知评估和根因定位。

4. 用户感知功能：
   - 定期对全网所有用户进行排查，找出感知差的用户，并分析导致问题的原因。

5. 语音业务质量指标MOS值分析：
   - 根据MOS值识别出语音质差区域。
   - 对质差区域提供问题定界处理建议，并提供闭环验证。

6. 呼叫和注册记录查询：
   - 可以进行用户注册、呼叫的统计，以便定位通话质量异常环节。

7. 本地日志分析：
   - 可以查看全网或特定地市的本地日志，展示事件失败原因。
   - 可以按不同维度（如网元、小区、终端、用户）进行统计和分析。

8. 日志管理：
   - 可以对日志数据进行清理和整理，提高工作效率。

9. 账号管理：
   - 用于维护账号信息，包括用户注册、登录等操作。

10. 角色管理：
    - 对不同角色的用户分配权限，如管理员、普通用户等。
    - 可以对特定角色进行权限设置和限制。

11. 资源监控：
    - 可以查看全网或特定地市的资源使用情况，包括网络带宽、流量等。
    - 可以提供告警管理功能，及时通知维护人员处理问题。

12. 系统日志保存：
   - 通过Gbase数据库或HDFS进行数据存储和备份。
   - 可以支持自动清理超过保存时间的记录。

13. 日志历史查询：
    - 可以查看全网或特定地市的日志记录，方便分析和定位问题。

总结来说，VMAX主要提供用户分析、一键投诉、区域感知、用户感知、语音业务质量指标MOS值分析、本地日志分析、账号管理、角色管理、资源监控、系统日志保存等功能。


In [14]:
question="只保留前3个功能"
result = qa({"question": question})  
print(f"回答：{result['answer']}")

回答：VMAX的主要功能包括：

1. 用户分析：通过用户IMSI或号码、用户群组等信息，定位语音/视频通话质量异常的环节。

2. 业务管理：提供用户分析帮助，便于运营维护人员定位具体问题。

3. 日志管理：支持日志保存和查询，方便管理和分析。

4. 资源监控：实时监控网络资源使用情况。

5. 告警管理：自动比对拨测结果，提高故障检测效率。

6. 系统对接：支持与北向接口、云化上网日志XDR等接口的连接和数据传输。

7. 语音业务优化：评估优化功能提供评估报告模板，帮助识别质量差区域并定位问题。


In [15]:
question="把上面的结果整理成一段话描述，100个字"
result = qa({"question": question})  
print(f"回答：{result['answer']}")

回答：ZXVMAX-S语音业务的主要功能包括：

1. 用户分析：通过用户IMSI或号码等维度，查询通话记录、注册情况、媒体数据和信令详情，并结合最近7天的统计结果定位异常问题。

2. 评估优化：提供评估报告模板，帮助运维人员快速定位问题并提出解决方案。

3. 日志管理：支持日志保存、查询、批量导入等功能，便于用户管理和分析日志信息。

4. 账号管理：允许用户自定义账号和角色权限。

5. 角色管理：为不同用户提供不同的访问权限。

6. 资源监控：实时监测网络性能指标如KPI等，并提供预警功能。

7. 本地日志分析：支持本地日志的保存、查询和历史记录查询。

8. 用户感知：通过用户注册失败率、呼叫失败率等指标，帮助运维人员了解服务质量。

9. 账号管理：允许用户自定义账号权限。

10. 角色管理：为不同用户提供不同的访问权限。

11. 本地日志分析：支持本地日志的保存和查询。

12. 日志管理：提供日志历史记录查询功能，方便用户了解日志信息。
